In [37]:
df_balanced=pd.read_csv('../data/processed/df_balanced.csv')

In [52]:
df_balanced.columns

Index(['Vehicle Type', 'Distance (km)', 'Is Weekend', 'Weather Code',
       'Traffic Code', 'Day of Week', 'Month', 'Origin_Bangalore',
       'Origin_Chennai', 'Origin_Delhi', 'Origin_Hyderabad', 'Origin_Jaipur',
       'Origin_Kolkata', 'Origin_Lucknow', 'Origin_Mumbai', 'Origin_Pune',
       'Destination_Bangalore', 'Destination_Chennai', 'Destination_Delhi',
       'Destination_Hyderabad', 'Destination_Jaipur', 'Destination_Kolkata',
       'Destination_Lucknow', 'Destination_Mumbai', 'Destination_Pune',
       'Delayed'],
      dtype='object')

In [39]:
import h2o
from h2o.automl import H2OAutoML

In [40]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,8 hours 26 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 19 days
H2O_cluster_name:,H2O_from_python_ishanporwal_8q1em3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.347 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [41]:
h2o_df = h2o.H2OFrame(df_balanced)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [42]:
h2o_df['Delayed'] = h2o_df['Delayed'].asfactor()


In [43]:
target = 'Delayed'
features = [col for col in h2o_df.columns if col != target]

In [44]:
train, test = h2o_df.split_frame(ratios=[.8], seed=42)


In [45]:
aml = H2OAutoML(max_models=20, seed=42)


In [46]:
aml.train(x=features, y=target, training_frame=train)


AutoML progress: |
03:11:19.952: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/5
# GBM base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [47]:
lb = aml.leaderboard
print(lb)

model_id                                                     auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_BestOfFamily_1_AutoML_3_20241222_31119  0.974024   0.158888  0.980549               0.0562698  0.215436  0.0464127
StackedEnsemble_AllModels_1_AutoML_3_20241222_31119     0.974023   0.158877  0.980535               0.0562698  0.215442  0.0464151
DRF_1_AutoML_3_20241222_31119                           0.970971   0.170362  0.978875               0.0561117  0.22096   0.0488232
GBM_grid_1_AutoML_3_20241222_31119_model_3              0.969446   0.180663  0.978061               0.0573749  0.224367  0.0503406
GBM_3_AutoML_3_20241222_31119                           0.967956   0.170596  0.977102               0.0562171  0.220661  0.0486913
GBM_grid_1_AutoML_3_20241222_31119_model_5              0.967832   0.171804  0.977041               0.0562697  0.221251  0.048952
GBM_4_AutoML_3_20241222_31119                           0.967775   0.172072  0.97702

In [48]:
leader_model = aml.leader

In [49]:
predictions = leader_model.predict(test)
print(predictions)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
  predict           p0        p1
        1  1.64865e-08  1
        1  1.47175e-05  0.999985
        1  6.16433e-06  0.999994
        1  1.98887e-08  1
        1  0.000476874  0.999523
        1  5.67187e-08  1
        1  1.18035e-07  1
        1  4.72171e-06  0.999995
        1  1.33473e-05  0.999987
        0  0.85256      0.14744
[4623 rows x 3 columns]



In [50]:
performance = leader_model.model_performance(test)
print(performance)

ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.04899662843157115
RMSE: 0.22135182048397783
LogLoss: 0.1637877969693335
AUC: 0.9739458754724863
AUCPR: 0.9799207885517516
Gini: 0.9478917509449727
Null degrees of freedom: 4622
Residual degrees of freedom: 4618
Null deviance: 6408.848401208462
Residual deviance: 1514.381970778457
AIC: 1524.381970778457

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9981205982926721
       0     1     Error    Rate
-----  ----  ----  -------  --------------
0      2307  0     0        (0.0/2307.0)
1      279   2037  0.1205   (279.0/2316.0)
Total  2586  2037  0.0604   (279.0/4623.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.998121     0.935906  6
max f2                       0.147344     0.92237   220
max f0point5                 0.998121     0.973337  6

In [51]:
# Save the best model
model_path = h2o.save_model(model=aml.leader, path="models", force=True)
print(f"Model saved at: {model_path}")


Model saved at: /Users/ishanporwal/Downloads/shipment_delay_prediction/notebooks/models/StackedEnsemble_BestOfFamily_1_AutoML_3_20241222_31119


In [58]:
model_path = "../models/StackedEnsemble_BestOfFamily_1_AutoML_3_20241222_31119"
model = h2o.load_model(model_path)

# Get the features used by the model
features = model._model_json['output']['names']

# Print the features
print("Features used by the model:")
print(features)

Features used by the model:
['Vehicle Type', 'Distance (km)', 'Is Weekend', 'Weather Code', 'Traffic Code', 'Day of Week', 'Month', 'Origin_Bangalore', 'Origin_Chennai', 'Origin_Delhi', 'Origin_Hyderabad', 'Origin_Jaipur', 'Origin_Kolkata', 'Origin_Lucknow', 'Origin_Mumbai', 'Origin_Pune', 'Destination_Bangalore', 'Destination_Chennai', 'Destination_Delhi', 'Destination_Hyderabad', 'Destination_Jaipur', 'Destination_Kolkata', 'Destination_Lucknow', 'Destination_Mumbai', 'Destination_Pune', 'Delayed']
